In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from functools import reduce

### Merging CSV files by county

In [ ]:

# Merge on 'name_and_fips'

# casino_count.csv
# 'name_and_fips',
# 'casino_point_count'
# 'GEOID'

casino = pd.read_csv('yourpath.csv')

# HT_SOB_count.csv
# 'name_and_fips',
# 'SOB_point_count'
# 'GEOID'

sob = pd.read_csv('yourpath.csv')

# censusdata.csv
# includes both nibrs and census variables
# 'name_and_fips',
# 'GEOID'
# 'HT_incident_count'
# 'County_State'
# 'Below Poverty Level Rate'
# 'Population Density'
# 'Median Household Income'

census = pd.read_csv('yourpath.csv')

#highway_count.csv
# 'name_and_fips',
# 'GEOID',
#'highway_count'
highway = pd.read_csv('yourpath.csv')

#airport_counts.csv
# 'name_and_fips',
# 'GEOID'
#'medlarge_airport_count'

airport = pd.read_csv('yourpath.csv')


#census2.csv
# 'GEOID'
# 'County_State'
# 'Below Poverty Level'
# 'Total Population'
# 'Median Household Income'
census2 = pd.read_csv('yourpath.csv')

In [ ]:
#merging dataframes, calling on only specific columns
#census 3193
#merged_df 
merged_df = census[['GEOID', 'name_and_fips', 'ALAND', 'HT_incident_count']].merge(
    highway[['GEOID', 'highway_count']], on='GEOID', how='left')
#1400
merged_df = merged_df.merge(
    airport[['GEOID', 'medlarge_airport_count']], on='GEOID', how='left')
#444
merged_df = merged_df.merge(
    sob[['GEOID', 'SOB_point_count']], on='GEOID', how='left')

merged_df = merged_df.merge(
    casino[['GEOID', 'casino_point_count']], on='GEOID', how='left')
merged_df = merged_df.merge(
    census2[['GEOID', 'County_State', 'Below Poverty Level', 'Total Population','Median Household Income']], on='GEOID', how='left')
merged_df.head()

In [ ]:
# Replace NaN with 0 in column 'A'
merged_df['medlarge_airport_count'] = merged_df['medlarge_airport_count'].fillna(0)
merged_df['highway_count'] = merged_df['highway_count'].fillna(0)
merged_df['HT_incident_count'] = merged_df['HT_incident_count'].fillna(0)
merged_df['casino_point_count'] = merged_df['casino_point_count'].fillna(0)
#calculate pov rate and population density
merged_df['Population Density'] = merged_df['Total Population']/merged_df['ALAND'] * 100
merged_df['Below Poverty Level Rate'] = merged_df['Below Poverty Level'] / merged_df['Total Population'] * 100
merged_df

### Initial Exploratory Data Analysis

In [ ]:
pca = merged_df[merged_df['SOB_point_count'].notna()]

In [ ]:
#first attempted pearson correlation matrices on original dataset (29 counties)
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Assuming 'pca' is your DataFrame containing the columns of interest
columns_of_interest = ['HT_incident_count', 'highway_count', 
                       'medlarge_airport_count', 'SOB_point_count', 
                       'casino_point_count', 'Median Household Income', 
                       'Population Density', 'Below Poverty Level Rate']

# Subset your DataFrame 'pca' to include only the columns of interest
subset_pca = pca[columns_of_interest]

# Pairwise plot using seaborn
sns.pairplot(subset_pca)
plt.show()

In [ ]:
#scatterplot for casinos against human trafficking incidents
# Filter out rows where 'County_State' is 'Los Angeles, CA'
casino_ht = pca[pca['HT_incident_count'] < 100]
# casino_ht
# Scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(casino_ht['HT_incident_count'], casino_ht['casino_point_count'])
plt.title('Scatter Plot: Casinos vs HT Incidents')
plt.xlabel('HT_incident_count')
plt.ylabel('casino_point_count')
plt.grid(True)
plt.show()

In [ ]:
# casinos against SOB frequency by county
Filter out rows where 'County_State' is 'Los Angeles, CA'
casino_sob = pca[pca['SOB_point_count'] < 80]
#casino_ht
# Scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(casino_sob['SOB_point_count'], casino_sob['casino_point_count'])
plt.title('Scatter Plot: Casinos vs SOBs')
plt.xlabel('SOB_point_count')
plt.ylabel('casino_point_count')
plt.grid(True)
plt.show()

In [ ]:
#-----------------scatterplots of HT incidents against all other data------------------
#removing major outliers
filtered_df = pca[(pca['HT_incident_count'] <= 100) & (pca['HT_incident_count'] != 0)]

# Fixed variable for comparison
fixed_variable = 'HT_incident_count'

# List of variables to plot against the fixed variable
variables = ['highway_count', 
                       'medlarge_airport_count', 'SOB_point_count', 
                       'casino_point_count', 'Median Household Income', 
                       'Population Density', 'Below Poverty Level Rate']

# Create a figure with 1 row and 7 columns
fig, axs = plt.subplots(1, len(variables), figsize=(20, 5))

# Iterate through each variable and create a scatter plot
for i, var in enumerate(variables):
    axs[i].scatter(filtered_df[fixed_variable], filtered_df[var])
    axs[i].set_title(f'{var}')
    axs[i].set_xlabel(fixed_variable)
    axs[i].set_ylabel(var)

# Adjust layout
plt.tight_layout()
plt.show()

#next steps: remove points with 0 human trafficking incidents

In [ ]:
# Scatter plot of number of highways against number of sobs by county
plt.figure(figsize=(8, 6))
plt.scatter(merged_df['HT_incident_count'], merged_df['highway_count'])
plt.title('Scatter Plot: Highways vs SOBs')
plt.xlabel('HT_incident_count')
plt.ylabel('highway_count')
plt.grid(True)
plt.show()

In [ ]:
# Scatter plot of HT incident by county versus povery level (not normalized for population)
plt.figure(figsize=(8, 6))
plt.scatter(merged_df['HT_incident_count'], merged_df['Below Poverty Level Rate'])
plt.title('Scatter Plot: HT Incidents vs Poverty Level')
plt.xlabel('HT_incident_count')
plt.ylabel('Below Poverty Level Rate')
plt.grid(True)
plt.show()